In [ ]:
from fenics import *
from multiphenics import *

In [ ]:
subdomains_infile = "../meshes/BrainWeb_Subject54_reduced.xdmf"
boundary_infile = "../meshes/BrainWeb_Subject54_boundaries.xdmf"

# subdomain ids
fluid_id = 2
porous_id = 1

# boundary ids
interface_id = 1
rigid_skull_id = 2
spinal_outlet_id = 3
fixed_stem_id = 4

infile = XDMFFile(subdomains_infile)
mesh = Mesh()
infile.read(mesh)
gdim = mesh.geometric_dimension()
subdomain_marker = MeshFunction("size_t", mesh, gdim, 0)
infile.read(subdomain_marker)
infile.close()

infile = XDMFFile(boundary_infile)
boundary_marker =  MeshFunction("size_t", mesh, gdim - 1, 0)
infile.read(boundary_marker)
infile.close()

In [ ]:
dxF = Measure("dx", domain=mesh, subdomain_data=subdomain_marker, subdomain_id=fluid_id)
dxP = Measure("dx", domain=mesh, subdomain_data=subdomain_marker, subdomain_id=porous_id)
dxD = Measure("dx", domain=mesh, subdomain_data=subdomain_marker)
dS = Measure("dS", domain=mesh, subdomain_data=boundary_marker)
ds = Measure("ds", domain=mesh, subdomain_data=boundary_marker)
ds_Sig = dS(interface_id)

In [ ]:
total_cfd_vol = assemble(1.0*dxF)
total_parenchyma_vol = assemble(1.0*dxP)
print(f"total_cfd_vol: {total_cfd_vol*1e3} l")
print(f"total_parenchyma_vol: {total_parenchyma_vol*1e3} l")

In [ ]:
total_intf_area = assemble(1.0*ds_Sig)
total_outer_area = assemble(1.0*ds(rigid_skull_id))
total_spinal_outlet_area = assemble(1.0*ds(spinal_outlet_id))
total_fixed_stem_area = assemble(1.0*ds(fixed_stem_id))

print(f"total_intf_area: {total_intf_area}")
print(f"total_outer_area: {total_outer_area}")
print(f"total_spinal_outlet_area: {total_spinal_outlet_area}")
print(f"total_fixed_stem_area: {total_fixed_stem_area}")


In [ ]:
def generate_subdomain_restriction(mesh, subdomains, subdomain_id):
    D = mesh.topology().dim()
    # Initialize empty restriction
    restriction = MeshRestriction(mesh, None)
    for d in range(D + 1):
        mesh_function_d = MeshFunction("bool", mesh, d)
        mesh_function_d.set_all(False)
        restriction.append(mesh_function_d)
    # Mark restriction mesh functions based on subdomain id
    for c in cells(mesh):
        if subdomains[c] == subdomain_id:
            restriction[D][c] = True
            for d in range(D):
                for e in entities(c, d):
                    restriction[d][e] = True
    # Return
    return restriction

In [ ]:
fluidrestriction = generate_subdomain_restriction(mesh, subdomain_marker, fluid_id)
porousrestriction = generate_subdomain_restriction(mesh, subdomain_marker, porous_id)

V = VectorFunctionSpace(mesh, "CG", 2)
W = FunctionSpace(mesh, "CG", 1)



In [ ]:
mesh.topology().dim()

In [ ]:
H = BlockFunctionSpace([V,W ], restrict=[fluidrestriction, porousrestriction])

In [ ]:
H.dim()